# <span style ='color:purple'>Import & settings

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%reload_ext autoreload
%autoreload 2

from data_provider.data_factory import data_provider
from tabular.dl_logs import logs_summary, exp_summary
from tabular.data import load_data
from tabular.tabular_models import compare_models_w_hpo

In [2]:
from types import SimpleNamespace
lt_logs_df = pd.read_csv(os.path.join('./logs/long_term_forecast.csv'), index_col = 0)
usc_args = SimpleNamespace(**lt_logs_df.loc[207].to_dict())

In [3]:
seq_len = 32
pred_len = seq_len
usc_args.seq_len = seq_len
usc_args.pred_len = pred_len
usc_args.scale = False

# <spn style = 'color:lightgreen'>DL Logs 

In [4]:
ldf = logs_summary(seq_len = seq_len)
ldf

test 4233


,model,features,fde,rmse
0,ST_GAT,MS,2.824565,1.696490
1,SABFormer,MS,2.849299,1.627774
2,Transformer,MS,2.931722,1.691967
3,iTransformer,S,2.937358,1.654779
4,Transformer,S,2.939580,1.653016
5,TimesNet,S,2.956967,1.665073
6,PatchTST,S,2.964681,1.676879
7,Nonstationary_Transformer,S,2.968956,1.694405
8,LSTM,MS,2.999465,1.800995
9,DLinear,S,3.018873,1.695828


In [5]:
edf = exp_summary([16,24,32,48,64,96])

test 5092
test 4640
test 4233
test 3515
test 2913
test 1965


In [6]:
idx = pd.IndexSlice
edf[['model','features']].merge(edf.round(2).loc[:,idx[:,'fde']], left_index=True, right_index=True).dropna(axis = 0)

seq_len,model,features,16_16,24_24,32_32,48_48,64_64,96_96,Average
metrics,,,fde,fde,fde,fde,fde,fde,fde
15,Transformer,MS,1.29,2.09,2.93,4.05,4.97,7.09,3.74
16,Transformer,S,1.30,2.19,2.94,4.10,5.18,7.28,3.83
11,ST_GAT,MS,1.38,2.29,2.82,4.14,5.20,7.27,3.85
7,Nonstationary_Transformer,S,1.33,2.21,2.97,4.17,5.19,7.31,3.86
18,iTransformer,S,1.29,2.15,2.94,4.30,5.24,7.29,3.87
9,PatchTST,S,1.37,2.24,2.96,4.17,5.24,7.25,3.88
3,DLinear,S,1.37,2.26,3.02,4.17,5.19,7.24,3.88
4,LSTM,MS,1.57,2.35,3.00,4.06,5.15,7.22,3.89
5,LSTM,S,1.54,2.37,3.04,4.16,5.20,7.28,3.93


# <span style ='color:cyan'> Linear Regression sensitivity analysis 

In [21]:
exps = []
seq_lens = [16,24,32,48,64,96]
groups = [list(range(1,9)), list(range(9,16))]
en_test = False

from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error
from scipy.stats import loguniform, uniform
hp = {'alpha':loguniform(1e-5, 1), 'l1_ratio':uniform(0,1)}

for seq_len in seq_lens:
    pred_len = seq_len
    exp = {}
    for i in range(7):
        if not 2**i > seq_len: 
            X_train, y_train, X_val, y_val, X_test, y_test = load_data(seq_len=seq_len, pred_len=pred_len, off_mask=True, groups=groups ,downsample_factor=2**i, verbose=False)
            print(f"Processing seq_len = {seq_len}, downsample_factor = {2**i}, n_groups = {len(groups)}, n_columns = {X_train.shape[1]}        ",end = '\r', flush = True )
            X_tr = np.concatenate((X_train, X_val))
            y_tr = np.concatenate((y_train, y_val))
            lr_model = LinearRegression().fit(X_tr,y_tr)
            y_pred_lr = lr_model.predict(X_test)
            if en_test :
                en_model = ElasticNet(max_iter=100000)
                rs = RandomizedSearchCV(en_model, hp, cv = 5 ,n_jobs=-1, n_iter = 256,refit=True).fit(X_tr, y_tr)
                y_pred_en = rs.predict(X_test)
                exp[2**i] = min(root_mean_squared_error(y_pred_lr, y_test),root_mean_squared_error(y_pred_en, y_test))
            else :
                exp[2**i] = root_mean_squared_error(y_pred_lr, y_test)
                
    exps.append(exp)
res_df = pd.DataFrame(exps, index = seq_lens, columns = [2**i for i in range(7)])

In [22]:
res_df.round(2)

,1,2,4,8,16,32,64
16,2.32,2.02,1.81,1.78,1.77,NaN,NaN
24,3.17,2.94,2.66,2.58,2.55,NaN,NaN
32,3.80,3.65,3.38,3.26,3.21,3.20,NaN
48,4.71,4.61,4.37,4.17,4.09,4.06,NaN
64,5.61,5.58,5.41,5.19,5.08,5.04,5.11
96,7.53,7.47,7.42,7.31,7.18,7.12,7.23


# <span style = 'color:pink'>Multi-model random search

In [19]:
groups = [list(range(1,9)), list(range(9,16))]
X_train, y_train, X_val, y_val, X_test, y_test = load_data(seq_len=seq_len, pred_len=pred_len, off_mask=True, groups=groups, downsample_factor=2)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((9167, 12), (9167,), (1964, 12), (1964,), (1965, 12), (1965,))

In [9]:
comp_df = compare_models_w_hpo(X_train=X_train, y_train=y_train,X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test,
                               cv = False, n_iter_ml=4, n_iter_ann=1, method='regression')

Tested Linear Regression - reached RMSE of 3.380 over the test set
Tested ElasticNet - reached RMSE of 3.770 over the test set
Tested SGD  - reached RMSE of 3.767 over the test set
Tested kNN - reached RMSE of 3.907 over the test set
Tested Support Vector - RBF - reached RMSE of 3.688 over the test set
Tested Decision Tree - reached RMSE of 3.504 over the test set


KeyboardInterrupt: 

In [ ]:
comp_df

,Best hyperparameters,RMSE
Model,,
Gradient Boosting,"{'learning_rate': 0.0035520547384062426, 'max_...",3.838167
Decision Tree,"{'max_depth': 1, 'min_samples_leaf': 28, 'min_...",3.849217
ElasticNet,"{'alpha': 0.0017646887788954836, 'l1_ratio': 0...",3.851696
Linear Regression,{},3.859684
Support Vector - RBF,"{'C': 15.708982413431276, 'gamma': 0.317939133...",3.860540
SGD,"{'alpha': 0.0002294533823356376, 'l1_ratio': 0...",3.870250
XGB,"{'eta': 0.49645042271838835, 'gamma': 718.6104...",3.904746
ADA Boost,"{'learning_rate': 0.0013291819648181275, 'n_es...",3.924627
Random Forest,"{'criterion': 'friedman_mse', 'max_depth': 9, ...",4.049795
